In [1]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 8.4 MB/s eta 0:00:00


In [2]:
!pip install transformers==4.28.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.6 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Check if GPU is available and set device accordingly
print("device : ", device)

device :  cuda


In [5]:
file_path = "/content/gdrive/My Drive/last_Dataset.csv"  
data = pd.read_csv(file_path)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15512 entries, 0 to 15511
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  15512 non-null  object
 1   lable    15512 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 242.5+ KB


In [7]:
# ㄹ
data = data.astype({"lable":"int"})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15512 entries, 0 to 15511
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  15512 non-null  object
 1   lable    15512 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 242.5+ KB


In [8]:
# 결측값 제거
data.drop_duplicates(subset=["content"], inplace = True)

In [9]:
# 전체 데이터를 학습+검증 데이터셋과 테스트 데이터셋으로 분할
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 학습+검증 데이터셋을 학습 데이터셋과 검증 데이터셋으로 분할
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=42)

In [11]:
# pre-trained KcELECTRA_base 모델 사용
MODEL_NAME = "beomi/KcELECTRA-base"
  
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [12]:
# 데이터 셋 토큰화
tokenized_train_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)
tokenized_val_sentences = tokenizer(
    list(val_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)
tokenized_test_sentences = tokenizer(
    list(train_data["content"]),
    return_tensors="pt",
    max_length = 128,
    padding = True,
    truncation = True,
    add_special_tokens = True,
)

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_label = train_data["lable"].values
val_label = val_data["lable"].values
test_label = test_data["lable"].values

# 각 데이터셋에 대한 CustomDataset 객체 생성
train_dataset = CustomDataset(tokenized_train_sentences, train_label)
val_dataset = CustomDataset(tokenized_val_sentences, val_label)
test_dataset = CustomDataset(tokenized_test_sentences, test_label)

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(device)

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.out_proj.weight', 'classifi

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [16]:
training_args = TrainingArguments(
    output_dir = './results',          # 출력 디렉토리 설정
    num_train_epochs = 3,             # epoch 횟수
    per_device_train_batch_size = 8,  # batch size per device during training
    per_device_eval_batch_size = 64,   # batch size for evaluation
    logging_dir = './logs',            # 로그 저장 디렉토리
    logging_steps = 100,
    save_total_limit = 2,  # 체크포인트 파일의 최대 개수
    evaluation_strategy = 'steps',  # 평가를 스텝마다 실행
    eval_steps = 100,  # 100스텝마다 평가 실행
)

In [17]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [18]:
trainer = Trainer(
    model = model,                         
    args = training_args,                  
    train_dataset = train_dataset,         
    eval_dataset = val_dataset,  
    compute_metrics = compute_metrics,       
)

In [ ]:
# 모델 학습 및 검증
trainer.train()

# 테스트 데이터셋에 대한 예측을 수행합니다.
predictions = trainer.predict(test_dataset)

# 예측 결과를 확률에서 하나의 클래스로 변환합니다.
pred_labels = np.argmax(predictions.predictions, axis=-1)

# 정밀도, 재현율, F1 점수를 계산합니다.
precision = precision_score(test_label, pred_labels, average='binary')
recall = recall_score(test_label, pred_labels, average='binary')
f1 = f1_score(test_label, pred_labels, average='binary')

# 정확도를 계산합니다.
acc = accuracy_score(test_label, pred_labels)

# 계산된 지표를 출력합니다.
print(f"Accuracy: {acc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

%load_ext tensorboard
%tensorboard --logdir ./logs

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
100,0.537400,0.404150,0.847872,0.882110,0.843405,0.924539
200,0.427700,0.421880,0.868085,0.897858,0.857817,0.941820
300,0.431200,0.373044,0.876950,0.904223,0.868045,0.943548
400,0.414600,0.341820,0.883333,0.905919,0.899489,0.912442
500,0.405400,0.330528,0.877305,0.905154,0.863494,0.951037
600,0.385100,0.436050,0.857447,0.876002,0.942895,0.817972
700,0.390500,0.367018,0.856383,0.876487,0.931303,0.827765


In [ ]:
# Test 
trainer.evaluate(eval_dataset = test_dataset)

In [ ]:
# Model 저장
model_path = "path/to/your/model/directory"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [ ]:
# 모델이 어느 정도 파악하는지 

def sentence_predict(sent):
  model.eval()

  tokenized_sent = tokenizer(
    sent,
    return_tensors="pt",
    max_length = 128,
    truncation = True,
    add_special_tokens = True,
  )

  # Move input tensors to the same device as the model
  tokenized_sent.to(device)

  with torch.no_grad():
    outputs = model(
        input_ids = tokenized_sent["input_ids"],
        attention_mask = tokenized_sent["attention_mask"],
        token_type_ids = tokenized_sent["token_type_ids"],
        )

    logits = outputs[0]
    logits = logits.detach().cpu()  # Move logits back to CPU for further processing
    result = logits.argmax(-1)
    if result == 0:
      result = ">> 악성 "
    elif result == 1:
      result = ">> 정상 "
    return result

while True:
  sentence = input("댓글을 입력하세요 :")
  if sentence == "0":
    break
  print(sentence_predict(sentence))
  print("\n")